# Strong Scaling of rotating Sphere (Stokes, steady)

We investigate: How much does the algorithm profit from more rescources for a fixed problem size? In theory the sequential part of the algorithm is the asymtotical limit, which is described by Ahmdal's law.

In [ ]:
//#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.IO;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Parameter sweeps
If you feel the urge to change something stick to this section ...
<br>Auxiliary datatype to map gridID onto controlobjects and control objects onto job settings.

In [ ]:
struct Parameterz{
    public Parameterz(int _Cores, int _Poly, int _Res){
        Cores = _Cores;
        Poly = _Poly;
        Res = _Res;
    }
    public int Cores;
    public int Poly;
    public int Res;
}

define job settings. One <code>Parameterz</code> marks one job setting (injective mapping).

|	 | DOF/cell | Res | cells | total DOF |
|----|:----:|:--:|:------:|:---------:|
| k2 | 34 	| 28 | 65,856 | 2,239,104 |
| k3 | 70 	| 22 | 31,944 | 2,236,080 |
| k4 | 125 	| 18 | 17,496 | 2,187,000 |

In [ ]:
// set parameterz
var Parameterz = new List<Parameterz>();

int[] CSweep = new int[] {128,256};
foreach(int cores in CSweep){
    // strong scaling
    Parameterz.Add(new Parameterz(cores,4,18));
    Parameterz.Add(new Parameterz(cores,3,22));
    Parameterz.Add(new Parameterz(cores,2,28));
}

int MemoryPerCore = 2000;

Define solver parameters

In [ ]:
bool useAMR = false;
bool useLoadBal = true;
int NoOfTimeSteps = 1;
bool Steady = true;
bool IncludeConvection = false;
var Gshape = Shape.Sphere;

## Init Database, Client and Workflowmanager
Set names of database and tables to be written out. 
Names are generated of environment variables (build information of jenkins).
There are defaults though, you see no need change anything.

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("BUILD_DISPLAY_NAME");
string buildNr = System.Environment.GetEnvironmentVariable("BUILD_NUMBER");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "run" : buildname;
string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
buildNr = String.IsNullOrEmpty(buildNr)? thedate : buildNr;

string database_name = String.IsNullOrEmpty(dbname)? "DB_Benchmarks" : dbname;
string table_name = String.Concat(buildname, "_", buildNr);

show available exection queues

In [ ]:
ExecutionQueues
//var myBatch = (MsHPC2012Client)ExecutionQueues[1];

index,type,Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,MonoDebug,AllowedDatabasesPaths,BatchInstructionDir,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.SlurmClient,jw52xeqa,<null>,lcluster14.hrz.tu-darmstadt.de,C:\Users\weber\.ssh\id_rsa,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=2000 ]",/home/jw52xeqa/Deployerie,special00006,<null>,X:\Deployerie,True,HHLR,dotnet,False,[ W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa ],,,,
1,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,,,,,,,,,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,,[ D:\trash_db == ],<null>,,,
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,FDY\weber,,DC2,,,,,,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,[ \\hpccluster\hpccluster-scratch\weber\trash_DB == ],,[ HPCCLUSTER2 ],Normal,True
3,BoSSS.Application.BoSSSpad.SlurmClient,jw52xeqa,<null>,lcluster5.hrz.tu-darmstadt.de,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,X:\Deployerie,True,<null>,dotnet,False,,,,,


Client setup and <code>\#SBATCH</code> configuration:
- <code>-N</code> (nodes),
- <code>-C</code> (Processor architecture),
- <code>--mem-per-cpu</code> (allocated memory per core).
<br> Note: <code>--mem-per-cpu</code> must be set that the job is accepted by Lichtenberg scheduler.

In [ ]:
var myBatch = (SlurmClient)GetDefaultQueue();
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -N 6", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+MemoryPerCore});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands

index,value
0,#SBATCH -N 6
1,#SBATCH -C avx512
2,#SBATCH --mem-per-cpu=2000


In [ ]:
string WFlowName = table_name;
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

Project name is set to 'run_11-1-2022'.
Creating database 'W:\work\scratch\jw52xeqa\run_11-1-2022'.


Set database

In [ ]:
var pair = myBatch.AllowedDatabasesPaths.Pick(0);
string DBName = @"\"+database_name;
string localpath=pair.LocalMountPath+DBName;
string remotepath=pair.PathAtRemote+DBName;
var myDB = OpenOrCreateDatabase(localpath);

Creating database 'W:\work\scratch\jw52xeqa\DB_Benchmarks'.


## Generate Grid
- Domain (-2,4)x(-1,1)x(-1,1)
- equidistant cells, resolution is chosen according to <code>Parameterz.Res</code>

In [ ]:
static double xMax = 4.0, yMax = 1.0, zMax = 1.0;
static double xMin = -2.0, yMin = -1.0,zMin = -1.0;

Some auxiliary methods: 
- calculate N_p (DOF of a scalar variable in 3D),
- optional: predefined partitioning (not used yet)

In [ ]:
static class Utils {
    // DOF per cell in 3D
    static public int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }
    static public Func<double[],int> GetPartFunc(int cores){
        Func<double[], int> MakeMyPartioning = delegate (double[] X) {
        double x  = X[0];
        double y  = X[1];
        double z  = X[2];
    
        int sx = 1;
        int sy = 1;
        int sz = 1;
        for (int i = 0; i < Math.Log(cores, 2); i++) {
            if (i % 3 == 0)
                sx*= 2;
            else if(i % 3 == 1)
                sy*=2;
            else
                sz*=2;
        }
    
    
        double xspan = (xMax - xMin) / sx;
        double yspan = (yMax - yMin) / sy;
        double zspan = (zMax - zMin) / sz;
        int rank     = int.MaxValue;
        int icore    = 0;
        for (int i = 0; i < sx; i++) {
            for (int j = 0; j < sy; j++) {
                for(int k=0;k<sz;k++){
                    bool xtrue = x <= xspan * (i + 1) + xMin;
                    bool ytrue = y <= yspan * (j + 1) + yMin;
                    bool ztrue = z <= zspan * (k + 1) + zMin;
                    if (xtrue && ytrue && ztrue) {
                        rank = icore;
                        return rank;
                    }
                    icore++;
                }
            }
        }
    
        return rank;
        };
    return MakeMyPartioning;
    }
}

generate all grids defined by <code>Parameterz.Res</code>. If grid already exists, continue. 

In [ ]:
var Grids = new Dictionary<int, IGridInfo>();
foreach(var P in Parameterz){
    int Res = P.Res;
    if(Grids.TryGetValue(Res,out IGridInfo ignore))
        continue;
    int Stretching = (int)Math.Floor(Math.Abs(xMax-xMin)/Math.Abs(yMax-yMin));
    //int Stretching = 1;
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Stretching*Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotBenchmarkGrid";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    //var tmp = myDB.Grids.Where(g=>g.Name.Equals(gname)&&g.NumberOfCells==Res*Res*Res); // this leads to exception in case of broken grids
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        Grids.Add(Res,tmp.Pick(0));
        continue;
    }
    
    grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
    grd.Name = gname;
    //grd.AddPredefinedPartitioning("bisec", GetPartFunc);

    grd.EdgeTagNames.Add(1, "Velocity_inlet");
    grd.EdgeTagNames.Add(2, "Wall");
    grd.EdgeTagNames.Add(3, "Pressure_Outlet");

    grd.DefineEdgeTags(delegate (double[] _X) {
        var X = _X;
        double x, y, z;
        x = X[0];
        y = X[1];
        z = X[2];
        if(Math.Abs(x-xMin)<1E-8)
            return 1;
        else
            return 3;
    });
    myDB.SaveGrid(ref grd,true);
    Grids.Add(Res,grd);
} Grids.Keys.ToList()

Grid Edge Tags changed.
Grid Edge Tags changed.
Grid Edge Tags changed.


index,value
0,18
1,22
2,28


## Generate Control object

### governing equations
- incompressible steady Navier-Stokes:
<br>$\frac{\partial \rho \vec{u}}{\partial t}+ \nabla \cdot ( \rho \vec{u} \otimes \vec{u}) + \nabla p - \eta \Delta \vec{u} = \vec{f} \ \ in \ \ \Omega_F(t) \times (0,T)$
<br>$\nabla \cdot \vec{u} = 0 \quad in \ \ \Omega_F(t) \times (0,T)$
- with inital conditions:
<br>$\vec{u}(\vec{x},0)  =  \vec{0} \quad on \ \ \Omega_F(0)$
<br>$p(\vec{x},0)  =  0 \quad  on \ \ \Omega_F(0)$
- with boundary conditions:
<br>$\vec{u}(\vec{x},t)  =  \vec{u}_{Inlet} \ \  on \ \ \Gamma_{Inlet} = \{ \vec{X} \in \partial \Omega_F(t) |  x=-2 \}$
<br>$p \mathbf{I} - \frac{1}{Re} \nabla \vec{u} \vec{n}_{ \Gamma_{pOut} } = 0 \ \ on \ \ \Gamma_{pOut} = \partial \Omega \backslash \Gamma_{Inlet} $ 
<br>$\vec{u}(\vec{x},t) = \boldsymbol{\omega}(t) \times \vec{r} \quad on \ \ \mathcal{J} = \partial \Omega_S \cap \partial \Omega_F$

- Inlet-Velocity $u_{Inlet}=\frac{Re*\mu_A}{\rho_A*d_{hyd}}$
- angular velocity of rotating sphere $\boldsymbol{\omega}(t)=\frac{Re*\mu_A}{\rho_A*d_{hyd}*1m}$

### Notes:
- for simplicity we stick to a steady linear problem (we do not have to worry about time stepping and NL-solver at this point)
- why sphere? answer: we do not have to enforce continuity of Levelset. Although sharp edges demand higher local resolution (skip AMR at this point)

In [ ]:
int SpaceDim = 3;

In [ ]:
Func<IGridInfo, int, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k){
    XNSE_Control C = new XNSE_Control();
    // basic database options
    // ======================
    C.AlternateDbPaths = new[] {
        (localpath, ""),
        (remotepath,"")};
    C.savetodb = true;
    int J  = grd.NumberOfCells;
    C.SessionName = string.Format("J{0}_k{1}_t{2}", J, k,NoOfTimeSteps);
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");
    C.Tags.Add("reortho_Iter2_sameRes");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 1;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.clusterHilbert;
    C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = useLoadBal;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 1;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;

    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 50;
    double muA = 1;
    
    double partRad = 0.3001;
    double d_hyd = 2*partRad;
    double anglev = Re*muA/rhoA/(d_hyd); // angular velocity [1/sec]
    //double anglev = 0.0;
    double VelocityIn = Re*muA/rhoA/d_hyd;
    double[] pos = new double[SpaceDim];

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim);
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis("x");

    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");
    C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X) => {VelocityIn}"));

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = 0.1;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.LinearSolver.NoOfMultigridLevels = 4;
    C.LinearSolver.ConvergenceCriterion = 1E-6;
    C.LinearSolver.MaxSolverIterations = 500;
    C.LinearSolver.MaxKrylovDim = 50;
    C.LinearSolver.TargetBlockSize = 1000;
    C.LinearSolver.verbose = true;
    C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-6;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = useAMR;
    if (useAMR) {
        C.SetMaximalRefinementLevel(1);
        C.AMR_startUpSweeps = 0;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = 0.1;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler;
    C.dtFixed = dt;
    return C;
};

In [ ]:
var controls = new Dictionary<Parameterz,XNSE_Control>();
foreach(var P in Parameterz){
    int k = P.Poly;
    Grids.TryGetValue(P.Res,out IGridInfo grd);
    controls.Add(P,GenXNSECtrl(grd,k));
} controls.Values.Select(s=>s.SessionName)

index,value
0,J17496_k4_t1_Stokes
1,J31944_k3_t1_Stokes
2,J65856_k2_t1_Stokes
3,J17496_k4_t1_Stokes
4,J31944_k3_t1_Stokes
5,J65856_k2_t1_Stokes


## Submit & Run Jobs at Server

mapping controlfiles to jobconfiguration alias number of cores in particular

In [ ]:
controls.Select(s=>s.Value.SessionName)

index,value
0,J17496_k4_t1_Stokes
1,J31944_k3_t1_Stokes
2,J65856_k2_t1_Stokes
3,J17496_k4_t1_Stokes
4,J31944_k3_t1_Stokes
5,J65856_k2_t1_Stokes


In [ ]:
int iSweep=0;
foreach(var ctrl in controls){
    try{
    int cores= ctrl.Key.Cores;
    var ctrlobj = ctrl.Value;
    string sessname = ctrlobj.SessionName;
    ctrlobj.SessionName = sessname + "_c"+cores+"_Re50_"+"mue_"+ctrlobj.PhysicalParameters.mu_A;
    var aJob   = new Job("rotSphereInlet_"+Gshape+ctrlobj.SessionName,typeof(XNSE));
    aJob.SetControlObject(ctrlobj);
    aJob.NumberOfMPIProcs         = cores;
    aJob.ExecutionTime            = "3:00:00";
    aJob.UseComputeNodesExclusive = true;
    aJob.Activate(myBatch);
    iSweep++;
    } catch (Exception ex){
        Console.WriteLine(ex.Message);
    }
}

Deploying job rotSphereInlet_SphereJ17496_k4_t1_Stokes_c128_Re50_mue_1 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\run_11-1-2022-XNSE_Solver2022Jan11_174849
copied 49 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
24877684

Deploying job rotSphereInlet_SphereJ31944_k3_t1_Stokes_c128_Re50_mue_1 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\run_11-1-2022-XNSE_Solver2022Jan11_174902
copied 49 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.
24877687

Deploying job rotSphereInlet_SphereJ65856_k2_t1_Stokes_c128_Re50_mue_1 ... 
Deploying executables and additional files ...
Deployment directory: X:\Deployerie\run_11-1-2022-XNSE_Solver2022Jan11_174914
copied 49 files.
   written file: control.obj


Error: System.OperationCanceledException: Command :SubmitCode: int iSweep=0;
foreach(var ctrl in controls){
    t ... cancelled.

Wait until all jobs terminate. Checking in 60 sec intervals. Printing out every 15 min ...

In [ ]:
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(60,900)